# Part 1 - Reading in the data

## Part 1.1

In [395]:
import glob
import pandas as pd
import matplotlib.pyplot as plt

# Define the path to the data files
DATA_PATH = './data'
fn = f'{DATA_PATH}/SCE*.csv'

dfs=[]

# Loop through files and read them into a list of DataFrames
for i in glob.glob(fn):
    d = pd.read_csv(i, parse_dates=['date'],sep=';')
    dfs.append(d)

# Concatenate all DataFrames into a single DataFrame, and convert 'date' column to datetime
df_original = pd.concat(dfs)

## Part 1.2

In [396]:
def overview_stats(data):

    # Finding number of unique users
    unique_ids = data['userid'].nunique()
    print(f'Number of unique users: {unique_ids}')

    # Finding number of data rows
    data_rows = data.shape[0]
    print(f'Number of data rows: {data_rows}')

    # Finding number of unique waves
    unique_waves = data['wid'].nunique()
    print(f'Number of unique waves: {unique_waves}')

    # Finding date range
    first_date = data['date'].min()
    last_date = data['date'].max()

    # Printing first and last dates observed
    print(f'Data ranges from {first_date.date()} to {last_date.date()}')
    
    return None

overview_stats(df_original)

Number of unique users: 23369
Number of data rows: 176101
Number of unique waves: 139
Data ranges from 2013-06-01 to 2024-12-31


# Part 2

## Part 2.1

In [397]:
# Sort by 'userid' and 'date'
# Creating new dataframe (df) to avoid modifying original dataframe (df_original)

df = df_original.sort_values(["userid", "date"], ignore_index=True)

#Could have looped through columns but this is clearer
num_cols = [c for c in df.columns if "num_lit_" in c and "_correct" in c]

# Fill missing values with the first observed value per user
first_values_per_user = df.groupby("userid")[num_cols].transform("first")
df[num_cols] = df[num_cols].fillna(first_values_per_user)
df

,userid,wid,date,weight,female,educ,age,hispanic,black,couple,...,num_lit_q3,num_lit_q3_correct,num_lit_q5,num_lit_q5_correct,num_lit_q6,num_lit_q6_correct,num_lit_q8,num_lit_q8_correct,num_lit_q9,num_lit_q9_correct
0,70000220,201306,2013-06-04,16.3,1.0,3.0,28.0,0.0,1.0,0.0,...,100.0,0.0,100.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN
1,70000224,201306,2013-06-03,0.2,0.0,4.0,65.0,0.0,0.0,1.0,...,10.0,1.0,100.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN
2,70000234,201306,2013-06-17,4.1,1.0,3.0,41.0,0.0,0.0,1.0,...,10.0,1.0,100.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN
3,70000238,201306,2013-06-13,3.0,0.0,3.0,74.0,0.0,0.0,1.0,...,10.0,1.0,1.0,0.0,5.0,1.0,NaN,NaN,NaN,NaN
4,70000238,201307,2013-07-10,1.9,0.0,3.0,74.0,0.0,0.0,NaN,...,NaN,1.0,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176096,75025299,202412,2024-12-19,0.6,1.0,3.0,33.0,0.0,0.0,1.0,...,10.0,1.0,100.0,1.0,5.0,1.0,2.0,0.0,2.0,1.0
176097,75025320,202412,2024-12-05,0.8,1.0,4.0,56.0,1.0,0.0,0.0,...,10.0,1.0,100.0,1.0,5.0,1.0,3.0,1.0,2.0,1.0
176098,75025337,202412,2024-12-21,1.0,1.0,3.0,68.0,0.0,0.0,1.0,...,10.0,1.0,100.0,1.0,5.0,1.0,3.0,1.0,2.0,1.0
176099,75025373,202412,2024-12-09,2.4,1.0,2.0,58.0,0.0,0.0,0.0,...,10.0,1.0,100.0,1.0,1.0,0.0,3.0,1.0,2.0,1.0


## Part 2.2 (Need improvements)

In [398]:
# Dropping rows with missing values in specified subsets of columns
demo = ['age', 'female', 'educ']
expectation = ['inflation', 'house_price_change', 'prob_stocks_up']

subsets = [
    ("demo", demo),
    ("expectation", expectation),
    ("numeracy", num_cols)
]

for name, ss in subsets:
    before = df.shape[0]
    df = df.dropna(subset=ss)
    after = df.shape[0]
    print(f"Numbers before dropping NAs for {name}: {before}, after: {after}, dropped: {before - after}")

Numbers before dropping NAs for demo: 176101, after: 175233, dropped: 868
Numbers before dropping NAs for expectation: 175233, after: 173550, dropped: 1683
Numbers before dropping NAs for numeracy: 173550, after: 137576, dropped: 35974


## Part 2.3

In [399]:
# drop observations below 0.1th and above 99.9th percentile for each expectation variable
for col in expectation:

    # Calculate percentiles and find number of rows before dropping
    before = df.shape[0]
    low = df[col].quantile(0.001)
    high = df[col].quantile(0.999)

    # Filter data to keep only rows within the percentile range, and calculate number of dropped rows
    df = df.query(f"{col} > {low} and {col} < {high}")
    dropped = before - df.shape[0]

    # Print results
    print(f"{col.capitalize()}: 0.1th percentile = {low}, 99.9th percentile = {high}")
    print(f"Observations dropped for {col}: {dropped}\n")

Inflation: 0.1th percentile = -75.0, 99.9th percentile = 100.0
Observations dropped for inflation: 434

House_price_change: 0.1th percentile = -50.0, 99.9th percentile = 100.0
Observations dropped for house_price_change: 418

Prob_stocks_up: 0.1th percentile = 0.0, 99.9th percentile = 100.0
Observations dropped for prob_stocks_up: 2456



## Part 2.4

In [400]:
# Creating a dummy variable for bachelors degree
df['college'] = (df['educ'] == 4).astype(int)

## Part 2.5

In [401]:
num_correct = df.groupby("userid")[num_cols].first().sum(axis=1)

# value counts normalized to fractions
fraction = num_correct.value_counts(normalize=True).sort_index() * 100

# display results
summary = fraction.rename("percent").reset_index().rename(columns={"index": "num_correct"})
print(summary)

   num_correct    percent
0          0.0   0.136379
1          1.0   0.704626
2          2.0   2.369587
3          3.0   5.654052
4          4.0  10.472781
5          5.0  16.939425
6          6.0  27.423571
7          7.0  36.299579


In [402]:
#Create a new column num_lit_high (“high numerical literacy”) equal to 1 if an individual had
#more correct responses than the median, and 0 otherwise."
median_num_correct = num_correct.median()
num_lit_high = (num_correct > median_num_correct).astype(int)

# add it to the dataframe by userid
df['num_lit_high'] = df['userid'].map(num_lit_high)

df

,userid,wid,date,weight,female,educ,age,hispanic,black,couple,...,num_lit_q5,num_lit_q5_correct,num_lit_q6,num_lit_q6_correct,num_lit_q8,num_lit_q8_correct,num_lit_q9,num_lit_q9_correct,college,num_lit_high
35333,70057317,201504,2015-04-10,0.5,0.0,4.0,70.0,0.0,0.0,1.0,...,10.0,0.0,5.0,1.0,3.0,1.0,2.0,1.0,1,0
35334,70057321,201504,2015-04-05,0.5,1.0,4.0,72.0,0.0,0.0,0.0,...,100.0,1.0,5.0,1.0,3.0,1.0,2.0,1.0,1,1
35335,70057321,201505,2015-05-11,0.5,1.0,4.0,72.0,0.0,0.0,0.0,...,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,1,1
35336,70057321,201506,2015-06-20,0.4,1.0,4.0,72.0,0.0,0.0,0.0,...,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,1,1
35337,70057321,201507,2015-07-22,0.4,1.0,4.0,72.0,0.0,0.0,0.0,...,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176096,75025299,202412,2024-12-19,0.6,1.0,3.0,33.0,0.0,0.0,1.0,...,100.0,1.0,5.0,1.0,2.0,0.0,2.0,1.0,0,0
176097,75025320,202412,2024-12-05,0.8,1.0,4.0,56.0,1.0,0.0,0.0,...,100.0,1.0,5.0,1.0,3.0,1.0,2.0,1.0,1,0
176098,75025337,202412,2024-12-21,1.0,1.0,3.0,68.0,0.0,0.0,1.0,...,100.0,1.0,5.0,1.0,3.0,1.0,2.0,1.0,0,1
176099,75025373,202412,2024-12-09,2.4,1.0,2.0,58.0,0.0,0.0,0.0,...,100.0,1.0,1.0,0.0,3.0,1.0,2.0,1.0,0,0


## Part 2.6

In [403]:
#Use function to print overview statistics of cleaned data
overview_stats(df)

Number of unique users: 17598
Number of data rows: 134268
Number of unique waves: 117
Data ranges from 2015-04-02 to 2024-12-31


# Part 3

## Part 3.1

In [404]:
def analyze_expectations_by_group(data, group_var, group_labels):
    """
    Calculate average expectations by group.
    
    Parameters:
    - data: DataFrame containing the data
    - group_var: string, column name for grouping ('female', 'college', 'num_lit_high')
    - group_labels: tuple of strings for group labels
    """
    # Calculate means for each expectation variable by group
    means = data.groupby(group_var)[expectation].mean()
    
    # Rename index values to more descriptive labels
    means.index = group_labels
    
    # Round to 2 decimal places for cleaner output
    means = means.round(2)
    
    print(f"\nAverage Expectations by {group_var.capitalize()}:")
    return means

In [ ]:
# Calculate averages for each group
group_specs = [
    ('female', ('Male', 'Female')),
    ('college', ('Non-college', 'College')),    # Assuming 'college' means at least a bachelor's degree (educ = 4), as in task 2.4
    ('num_lit_high', ('Low numeracy', 'High numeracy'))
]

# Using function and looping through group specifications, then printing results
for group_var, labels in group_specs:
    results = analyze_expectations_by_group(df, group_var, labels)
    print(results) 


Average Expectations by Female:
        inflation  house_price_change  prob_stocks_up
Male         4.56                4.27           45.84
Female       7.09                6.21           38.30

Average Expectations by College:
             inflation  house_price_change  prob_stocks_up
Non-college       7.53                6.58           38.24
College           4.47                4.18           45.15

Average Expectations by Num_lit_high:
               inflation  house_price_change  prob_stocks_up
Low numeracy        6.96                6.23            38.9
High numeracy       3.96                3.62            47.3
